In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string

import emoji, re
import nltk
import torchtext

from nltk.corpus import stopwords
# nltk.download('wordnet')
# nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize



In [2]:
# NOTES :
# pos_tag for knowing VBG, N, ADJ, ADV, ...

In [3]:
df = pd.read_csv("singapore_airlines_reviews.csv")
df

,published_date,published_platform,rating,type,text,title,helpful_votes
0,2024-03-12T14:41:14-04:00,Desktop,3,review,We used this airline to go from Singapore to L...,Ok,0
1,2024-03-11T19:39:13-04:00,Desktop,5,review,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,2024-03-11T12:20:23-04:00,Desktop,1,review,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,2024-03-11T07:12:27-04:00,Desktop,5,review,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,2024-03-10T05:34:18-04:00,Desktop,2,review,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0
...,...,...,...,...,...,...,...
9995,2018-08-06T03:48:21-04:00,Desktop,5,review,First part done with Singapore Airlines - acce...,"Flew to NZ 1st half Singapore Airlines, 2nd ha...",1
9996,2018-08-05T22:50:29-04:00,Mobile,5,review,And again a great Flight with Singapore Air. G...,Best Airline,1
9997,2018-08-05T22:47:06-04:00,Desktop,5,review,"We flew business class from Frankfurt, via Sin...",Superb service on Singapore Airlines,1
9998,2018-08-05T20:32:03-04:00,Desktop,4,review,"As always, the A380 aircraft was spotlessly pr...",A Comfortable Fiight Spoiled by lack of adequa...,2


In [4]:
def find_unique_values(columns):
    print('Total Unique Values:\n')
    for col in columns:
        print(f"{col} => {len(df[col].unique())}")

columns = find_unique_values(df.columns)

Total Unique Values:

published_date => 9997
published_platform => 2
rating => 5
type => 1
text => 10000
title => 8477
helpful_votes => 27


In [5]:
df.isna().sum()

published_date        0
published_platform    0
rating                0
type                  0
text                  0
title                 1
helpful_votes         0
dtype: int64

In [6]:
def clean_data(df):
    drop_columns = ['published_date', 'published_platform', 'type', 'title', 'helpful_votes']

    df.drop(columns=drop_columns, inplace=True)
    
    # Title has 1 NaN value, need to fillna
    df = df.fillna("")

    df.to_csv('data_clean.csv', index=False)

clean_data(df)

In [7]:
df

,rating,text
0,3,We used this airline to go from Singapore to L...
1,5,The service on Singapore Airlines Suites Class...
2,1,"Booked, paid and received email confirmation f..."
3,5,"Best airline in the world, seats, food, servic..."
4,2,Premium Economy Seating on Singapore Airlines ...
...,...,...
9995,5,First part done with Singapore Airlines - acce...
9996,5,And again a great Flight with Singapore Air. G...
9997,5,"We flew business class from Frankfurt, via Sin..."
9998,4,"As always, the A380 aircraft was spotlessly pr..."


In [8]:
def tokenization(paragraph, lowercase=True, stop_words=True):
    '''
    Tokenize for 1 Paragraph
    '''
    # Remove Emoji
    def remove_emoji(text):
        emojis = re.compile("["
                        u"\U0001F600-\U0001F64F"
                        u"\U0001F300-\U0001F5FF"
                        u"\U0001F680-\U0001F6FF"
                        u"\U0001F1E0-\U0001F1FF"
                        u"\U00002500-\U00002BEF" 
                        u"\U00002702-\U000027B0"
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u"\U00010000-\U0010ffff"
                        u"\u2640-\u2642"
                        u"\u2600-\u2B55"
                        u"\u200d"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\ufe0f"
                        u"\u3030"
                        "]+", flags=re.UNICODE)
        return emojis.sub(r'', text)
    
    def get_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN 
    
    
    if lowercase: paragraph = paragraph.lower()

    # Remove Emoji
    paragraph = remove_emoji(paragraph)

    # Remove Punctuation
    paragraph = paragraph.translate(str.maketrans('', '', string.punctuation))

    tokens = word_tokenize(paragraph)

    # tokens = paragraph.split()

    # Remove Stop Words
    if stop_words:
        stops = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stops]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(word=token, pos=get_pos(tag)) for token, tag in pos_tag(tokens)]

    # lemmatized_tokens = []
    # for token, tag in pos_tag(tokens):
    #     pos = get_pos(tag)
    #     print(f"{tag} : {pos}")
    #     lemmatized_token = lemmatizer.lemmatize(token, pos=pos)
    #     print(f"{token} : {lemmatized_token}")
    #     lemmatized_tokens.append(lemmatized_token)



    # print(tokens)
    return tokens



paragraphs = df['text']

tokenized_data = []

for paragraph in paragraphs:
    text = tokenization(paragraph, lowercase=True, stop_words=True)
    tokenized_data.append(text)



In [9]:
# from nltk import pos_tag
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet

# def get_wordnet_pos(treebank_tag):
#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN  # Default to noun if no match

# word = "leading"
# word_tokenized = word_tokenize(word)
# pos_tagged_word = pos_tag(word_tokenized)
# print(pos_tagged_word)

# lemmatizer = WordNetLemmatizer()
# lemmatized_word = lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos_tagged_word[0][1]))

# print(lemmatized_word)


In [4]:
import torchtext
glove = torchtext.vocab.GloVe(name="840B", dim=300)

100%|█████████▉| 2196016/2196017 [02:43<00:00, 13423.60it/s]
